In [1]:
!wget https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt

--2023-02-13 22:32:04--  https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘input.txt’

input.txt               [ <=>                ] 140.02K  --.-KB/s    in 0.1s    

2023-02-13 22:32:05 (1.27 MB/s) - ‘input.txt’ saved [143380]

